#Data processing

In [ ]:
from google.colab import files
api_token = files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle competitions download -c commonlitreadabilityprize

  0% 0.00/1.13M [00:00<?, ?B/s]
100% 1.13M/1.13M [00:00<00:00, 38.1MB/s]
  0% 0.00/6.79k [00:00<?, ?B/s]
100% 6.79k/6.79k [00:00<00:00, 6.89MB/s]
  0% 0.00/108 [00:00<?, ?B/s]
100% 108/108 [00:00<00:00, 92.7kB/s]


In [ ]:
!unzip /content/train.csv.zip

Archive:  /content/train.csv.zip
  inflating: train.csv               


In [ ]:
!pip install contractions
!pip install textstat

     |████████████████████████████████| 284 kB 13.6 MB/s 
     |████████████████████████████████| 321 kB 52.4 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85446 sha256=26fde7288fdf84e45e47535cc8845f337770a96c2759a66a2ae6e50cd24c458b
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
     |████████████████████████████████| 101 kB 6.9 MB/s 
     |████████████████████████████████| 2.0 MB 47.8 MB/s 


In [ ]:
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
# from sentence_transformers import SentenceTransformer
from collections import Counter
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.svm import SVR
from scipy.stats import loguniform
import csv
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
from tabulate import tabulate
import pandas as pd
import re
import datetime
import numpy as np
import pickle
import contractions
import textstat
import bs4
import requests
import json
import urllib
import pandas as pd
import os
import time
import re
import numpy as np
import nltk
import warnings
from nltk.tokenize import word_tokenize,sent_tokenize

tqdm.pandas()
pd.options.display.max_colwidth = None
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
data = pd.read_csv("train.csv")

In [ ]:
# we need only excerpt and target columns
data = data[['excerpt','target']]

In [ ]:
data['target'] = data['target'].astype("float32")

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2834 entries, 0 to 2833
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   excerpt  2834 non-null   object 
 1   target   2834 non-null   float32
dtypes: float32(1), object(1)
memory usage: 33.3+ KB


In [ ]:
X = data['excerpt']
y = data['target']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [ ]:
X_train = X_train.to_frame(name='excerpt')
X_test = X_test.to_frame(name='excerpt')

In [ ]:
# remove everythoing except a-z A-Z . and space
regex_only_alpha_space_dot = re.compile('[^a-zA-Z. ]')

# remove extra spaces
regex_multiple_space = re.compile(" +")

def pre_process_text(text):
  text = contractions.fix(text)
  text = regex_only_alpha_space_dot.sub(' ',text)
  text = regex_multiple_space.sub(" ",text)
  return text.lower()

In [ ]:
X_train['excerpt']= X_train['excerpt'].apply(pre_process_text)
X_test['excerpt'] = X_test['excerpt'].apply(pre_process_text)

In [ ]:
doc_lens = []
sent_lens =[]

for text in X_train['excerpt'].values:
  sentences = sent_tokenize(text)
  doc_lens.append(len(sentences))
  for sentence in sentences:
    words = word_tokenize(sentence)
    sent_lens.append(len(words))

In [ ]:
print("sentence length percentile :")
for i in range(0,105,5):
  print(i,np.percentile(sent_lens,i))

sentence length percentile :
0 1.0
5 6.0
10 8.0
15 9.0
20 10.0
25 12.0
30 13.0
35 14.0
40 15.0
45 17.0
50 18.0
55 19.0
60 21.0
65 23.0
70 25.0
75 27.0
80 30.0
85 33.0
90 38.0
95 46.0
100 152.0


In [ ]:
print("doc length percentile :")
for i in range(0,105,5):
  print(i,np.percentile(doc_lens,i))

doc length percentile :
0 2.0
5 4.0
10 5.0
15 6.0
20 6.0
25 6.0
30 7.0
35 7.0
40 7.0
45 8.0
50 8.0
55 8.0
60 9.0
65 9.0
70 10.0
75 10.0
80 11.0
85 12.0
90 14.0
95 16.0
100 34.0


In [ ]:
MAX_SENT_LEN = 46 #number of words
MAX_DOC_LEN = 16 # number of sentence

In [ ]:
def calculate_doc_features(doc):

  def calculate_other_doc_features(doc):
    words = word_tokenize(doc)
    num_sents = len(sent_tokenize(doc))
    num_words = len(words)


    num_word_longer_than_6_char = 0
    for word in words:
      if len(word) > 6:
        num_word_longer_than_6_char += 1

    lix = (num_word_longer_than_6_char / num_words ) + (num_words / num_sents)
    rix = num_word_longer_than_6_char / num_sents
    lexical_diversity = len(set(words)) / num_words

    return [lix,rix,lexical_diversity]

  text_stat_features = [textstat.flesch_reading_ease(doc),
                        textstat.flesch_kincaid_grade(doc),
                        textstat.automated_readability_index(doc),
                        textstat.coleman_liau_index(doc),
                        textstat.gunning_fog(doc),
                        textstat.smog_index(doc),
                        textstat.dale_chall_readability_score(doc)
                        ]
  text_stat_features.extend(calculate_other_doc_features(doc))

  return np.array(text_stat_features)

In [ ]:
def calculate_sent_features(doc,max_doc_len):

  sentences = sent_tokenize(doc)
  sent_features = []
  for sent in sentences:

    words = word_tokenize(sent)

    num_words = len(words)

    char_per_word = [len(word) for word in words]
    avg_char_per_word = sum(char_per_word) / len(char_per_word)

    num_long_words = len([word for word in words if len(word) > 6])

    sent_features.append([num_words,avg_char_per_word,num_long_words])

  sent_features = np.array(sent_features)
  if len(sent_features) > max_doc_len:
    sent_features = sent_features[:max_doc_len,:]
  else:
    pad_value = doc_pad_value = np.zeros((max_doc_len - sent_features.shape[0],
                                          sent_features.shape[1]
                                          )
                                       )

    sent_features = np.concatenate((sent_features,pad_value),axis=0)
  return np.array(sent_features)

In [ ]:
d = "When the young people returned to the ballroom, it presented a decidedly changed appearance. Instead of an interior scene, it was a winter landscape.\nThe floor was covered with snow-white canvas, not laid on smoothly, but rumpled over bumps and hillocks, like a real snow field. The numerous palms and evergreens that had decorated the room, were powdered with flour and strewn with tufts of cotton, like snow. Also diamond dust had been lightly sprinkled on them, and glittering crystal icicles hung from the branches.\nAt each end of the room, on the wall, hung a beautiful bear-skin rug.\nThese rugs were for prizes, one for the girls and one for the boys. And this was the game.\nThe girls were gathered at one end of the room and the boys at the other, and one end was called the North Pole, and the other the South Pole. Each player was given a small flag which they were to plant on reaching the Pole.\nThis would have been an easy matter, but each traveller was obliged to wear snowshoes."

print(calculate_doc_features(d).shape)
print(calculate_sent_features(d,MAX_DOC_LEN).shape)

(10,)
(16, 3)


In [ ]:
X_train['sent_feat'] = X_train['excerpt'].apply(calculate_sent_features,args=(MAX_DOC_LEN,))
X_test['sent_feat'] = X_test['excerpt'].apply(calculate_sent_features,args=(MAX_DOC_LEN,))

In [ ]:
X_train['doc_feat'] = X_train['excerpt'].apply(calculate_doc_features)
X_test['doc_feat'] = X_test['excerpt'].apply(calculate_doc_features)

In [ ]:
class DocTokenizer:
    def __init__(self,excerpts,max_doc_len,max_sent_len):

        self.max_doc_len = max_doc_len
        self.max_sent_len = max_sent_len

        # will use tokenizer for 
        self.excerpt_tokenizer = Tokenizer(oov_token="oov_token")
        self.excerpt_tokenizer.fit_on_texts(excerpts)
        self.vocab_size = len(self.excerpt_tokenizer.word_index) + 1
         
    def __call__(self, docs):
        output = []

        for doc in docs:
          sentences = sent_tokenize(doc)
          tokens = self.excerpt_tokenizer.texts_to_sequences(sentences)
          
          # 0 padded sentence
          padded_tokens = pad_sequences(tokens,maxlen=self.max_sent_len,padding='post')

          # 0 padded doc
          # print(self.max_doc_len, padded_tokens.shape[0])
          padded_doc = []
          if self.max_doc_len > padded_tokens.shape[0]:
            doc_pad_value = np.zeros(
                (self.max_doc_len - padded_tokens.shape[0], self.max_sent_len)
                )
            padded_doc = np.concatenate((padded_tokens,doc_pad_value),axis=0)
            
          else:
            padded_doc = padded_tokens[:self.max_doc_len,:]

          output.append(padded_doc)

        return np.array(output)

In [ ]:
# https://towardsdatascience.com/implementing-custom-data-generators-in-keras-de56f013581c
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, tokens,sent_feat,doc_feat,y,batch_size=32,shuffle=True):
        self.tokens = tokens
        self.sent_feat = sent_feat
        self.doc_feat = doc_feat
        self.y = y
        self.batch_size = batch_size
        self.indices = list(range(self.tokens.shape[0]))
        self.shuffle = shuffle
        self.on_epoch_end()
        

    def __len__(self):
        return len(self.indices) // self.batch_size

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        batch_tokens, batch_sent_feat, batch_doc_feat, batch_y = self.__get_data(batch)
        return [batch_tokens, batch_sent_feat, batch_doc_feat], batch_y

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

    def __get_data(self, batch):

        batch_tokens = []
        batch_sent_feat = []
        batch_doc_feat = []
        batch_y = []
        for id in batch:
            batch_tokens.append(self.tokens[id])
            batch_sent_feat.append(self.sent_feat[id])
            batch_doc_feat.append(self.doc_feat[id])
            batch_y.append(self.y[id])

        return [np.array(batch_tokens),
                np.array(batch_sent_feat),
                np.array(batch_doc_feat),
                np.array(batch_y)]

In [ ]:
tokenizer = DocTokenizer(X_train['excerpt'],MAX_DOC_LEN,MAX_SENT_LEN)
X_train_tokens= tokenizer(X_train['excerpt'])
X_test_tokens = tokenizer(X_test['excerpt'])

In [ ]:
print("X_train_tokens shape :",X_train_tokens.shape)
print("X_test_tokens shape :",X_test_tokens.shape)

X_train_tokens shape : (2267, 16, 46)
X_test_tokens shape : (567, 16, 46)


In [ ]:
X_train_sent_feat = np.array(X_train['sent_feat'].tolist())
X_test_sent_feat = np.array(X_test['sent_feat'].tolist())

In [ ]:
X_train_doc_feat = np.array(X_train['doc_feat'].tolist())
X_test_doc_feat = np.array(X_test['doc_feat'].tolist())

In [ ]:
batch_size = 32

In [ ]:
train_data_generator = DataGenerator(X_train_tokens,
                                     X_train_sent_feat,
                                     X_train_doc_feat,
                                     y_train.values,
                                     batch_size = batch_size)

test_data_generator = DataGenerator(X_test_tokens,
                                    X_test_sent_feat,
                                    X_test_doc_feat,
                                    y_test.values,
                                    batch_size = batch_size)

# ReadNet Model

Ref:<br>
https://www.tensorflow.org/text/tutorials/transformer <br>
https://www.kaggle.com/renjithrrkj/readnet-implemetation

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [ ]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

In [ ]:
class AttentionAggregation(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        self.query = tf.keras.layers.Dense(1,use_bias=False,activation='softmax')

    def call(self, x):  # (b, s, m)
        attns = self.query(x)
        enc = tf.matmul(tf.transpose(attns,perm=[0,2,1]), x)  # (b, 1, m)

        return tf.squeeze(enc,1)


class LinTanh(tf.keras.layers.Layer):
    def __init__(self, d_model):
        super().__init__()
        self.lintanh = tf.keras.layers.Dense(d_model,use_bias=False,activation='tanh')

    def call(self, x):
        return self.lintanh(x)


class LinFeatConcat(tf.keras.layers.Layer):
    def __init__(self, n_out):
        super().__init__()
        self.lin = tf.keras.layers.Dense(n_out, use_bias=False)
    def call(self, x, feats):
        return self.lin(tf.concat([x, feats], axis=1))


In [ ]:
class SentenceEncoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(SentenceEncoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

    self.lin_tanh = LinTanh(d_model=d_model)
    self.attn_agg = AttentionAggregation()
    self.lin_feat_concat = LinFeatConcat(n_out=self.d_model)

  def call(self, inputs, training=False, mask=None):
    x = inputs[0]
    sent_feat = inputs[1]
    seq_len = tf.shape(x)[2]

    x = self.embedding(x)  #(batch_size x doc_len x sent_len x embed_size)

    x_shape = tf.shape(x)
    x = tf.reshape(x,(x_shape[0]*x_shape[1],
                      x_shape[2],
                      x_shape[3]
                      )
                  ) # (batch_size*doc_len, sent_len, d_model)

    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    
    # adding embedding and position encoding.
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    x = self.lin_tanh(x)
    x = self.attn_agg(x)

    
    sent_feat = tf.reshape(sent_feat,(x_shape[0]*x_shape[1],-1))

    x = self.lin_feat_concat(x, sent_feat) 

    return tf.reshape(x,(x_shape[0],x_shape[1],x_shape[-1])) # (batch_size,doc_len,embedding)
    

In [ ]:
class DocEncoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(DocEncoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

    self.lin_tanh = LinTanh(d_model=d_model)
    self.attn_agg = AttentionAggregation()
    self.lin_feat_concat = LinFeatConcat(n_out=self.d_model)

  def call(self, inputs, training=False, mask=None):
    x = inputs[0]
    doc_feat = inputs[1]
    seq_len = tf.shape(x)[1]

    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    x = self.lin_tanh(x)
    x = self.attn_agg(x)

    # sent_feat = tf.reshape(sent_feat,(x_shape[0]*x_shape[1],-1))
    x = self.lin_feat_concat(x, doc_feat)

    return x # (b,embedding)
    

In [ ]:
class ReadNet(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(ReadNet, self).__init__()

    # get sentence features from words
    self.S_en = SentenceEncoder(num_layers= num_layers,
                          d_model=d_model,
                          num_heads=num_heads,
                          dff=dff,
                          input_vocab_size=input_vocab_size,
                          maximum_position_encoding=maximum_position_encoding
                          )

    # get doc lever features from sentence features
    self.D_en = DocEncoder(num_layers= num_layers,
                     d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     input_vocab_size=input_vocab_size,
                     maximum_position_encoding=maximum_position_encoding
                     )

    # output layer
    self.final_layer = tf.keras.layers.Dense(1)
  
  def call(self,inputs,training=False):

    x = inputs[0]
    sent_features = inputs[1]
    doc_features = inputs[2]

    x = self.S_en([x,sent_features],training=training)

    x = self.D_en([x,doc_features],training=training)

    op = self.final_layer(x)

    return op


In [ ]:
model = ReadNet(num_layers=6, d_model=512, num_heads=8,
                         dff=1024, input_vocab_size=tokenizer.vocab_size,
                         maximum_position_encoding=512)

In [ ]:
train_step_per_epoch = train_data_generator.__len__()
test_step_per_epoch = test_data_generator.__len__()

In [ ]:
checkpoint_filepath = "/content/drive/MyDrive/casestudy/CommonReadability/readnet_tf/checkpoint"

callback_earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=15,
                                                      mode='auto',
                                                      restore_best_weights=True
                                                      )

callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath,
                                                         monitor='val_loss',
                                                         save_best_only=True,
                                                         save_weights_only=False,
                                                         mode='auto',
                                                         save_freq='epoch'
                                                         )

callback_reducelr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                         factor=0.2,
                                                         patience=5
                                                         )

In [ ]:
# https://towardsdatascience.com/creating-custom-loss-functions-using-tensorflow-2-96c123d5ce6c
def rmse_loss(y_true, y_pred):
    #difference between true label and predicted label
    error = y_true-y_pred    
    #square of the error
    sqr_error = K.square(error)
    #mean of the square of the error
    mean_sqr_error = K.mean(sqr_error)
    #square root of the mean of the square of the error
    sqrt_mean_sqr_error = K.sqrt(mean_sqr_error)
    #return the error
    return sqrt_mean_sqr_error

model.compile(optimizer='adam',
              loss=rmse_loss,
              )

In [ ]:
history = model.fit_generator(train_data_generator,
                              steps_per_epoch=train_step_per_epoch,
                              epochs = 100,
                              validation_data = test_data_generator,
                              validation_steps = test_step_per_epoch,
                              callbacks = [callback_checkpoint,
                                           callback_reducelr,
                                           callback_earlystop
                                           ]
                              )

Epoch 1/100
70/70 [==============================] - 147s 2s/step - loss: 22.4166 - val_loss: 15.5417


INFO:tensorflow:Assets written to: /content/drive/MyDrive/casestudy/CommonReadability/readnet_tf/checkpoint/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/casestudy/CommonReadability/readnet_tf/checkpoint/assets


Epoch 2/100
70/70 [==============================] - 125s 2s/step - loss: 9.4762 - val_loss: 5.0232


INFO:tensorflow:Assets written to: /content/drive/MyDrive/casestudy/CommonReadability/readnet_tf/checkpoint/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/casestudy/CommonReadability/readnet_tf/checkpoint/assets


Epoch 37/100
70/70 [==============================] - 124s 2s/step - loss: 0.8360 - val_loss: 0.8258
Epoch 38/100
70/70 [==============================] - 123s 2s/step - loss: 0.8340 - val_loss: 0.8220


INFO:tensorflow:Assets written to: /content/drive/MyDrive/casestudy/CommonReadability/readnet_tf/checkpoint/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/casestudy/CommonReadability/readnet_tf/checkpoint/assets


Epoch 39/100
70/70 [==============================] - 124s 2s/step - loss: 0.8305 - val_loss: 0.8260
Epoch 40/100
70/70 [==============================] - 123s 2s/step - loss: 0.8420 - val_loss: 0.8279
Epoch 41/100
70/70 [==============================] - 123s 2s/step - loss: 0.8312 - val_loss: 0.8438
Epoch 42/100
70/70 [==============================] - 123s 2s/step - loss: 0.8289 - val_loss: 0.8380
Epoch 43/100
70/70 [==============================] - 123s 2s/step - loss: 0.8386 - val_loss: 0.8453
Epoch 44/100
70/70 [==============================] - 123s 2s/step - loss: 0.8257 - val_loss: 0.8340
Epoch 45/100
70/70 [==============================] - 123s 2s/step - loss: 0.8235 - val_loss: 0.8623
Epoch 46/100
70/70 [==============================] - 124s 2s/step - loss: 0.8355 - val_loss: 0.8348
Epoch 47/100
70/70 [==============================] - 124s 2s/step - loss: 0.8286 - val_loss: 0.8326
Epoch 48/100
70/70 [==============================] - 124s 2s/step - loss: 0.8232 - val_los

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['loss'][1:])
plt.plot(history.history['val_loss'][1:])
plt.title('model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
tf.keras.models.save_model(model,"Test")

INFO:tensorflow:Assets written to: Test/assets


INFO:tensorflow:Assets written to: Test/assets


In [ ]:
m = tf.keras.models.load_model("Test",custom_objects={'rmse_loss': rmse_loss})

In [ ]:
print("Train RMSE",m.evaluate_generator(train_data_generator))
print("Test RMSE",m.evaluate_generator(test_data_generator))

Train RMSE 0.8216878771781921
Test RMSE 0.8258156776428223
